In [1]:
# Setting the environment variables

In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook --no-browser"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

# Ecommerce Churn Assignment

The aim of the assignment is to build a model that predicts whether a person purchases an item after it has been added to the cart or not. Being a classification problem, you are expected to use your understanding of all the three models covered till now. You must select the most robust model and provide a solution that predicts the churn in the most suitable manner. 

For this assignment, you are provided the data associated with an e-commerce company for the month of October 2019. Your task is to first analyse the data, and then perform multiple steps towards the model building process.

The broad tasks are:
- Data Exploration
- Feature Engineering
- Model Selection
- Model Inference

### Data description

The dataset stores the information of a customer session on the e-commerce platform. It records the activity and the associated parameters with it.

- **event_time**: Date and time when user accesses the platform
- **event_type**: Action performed by the customer
            - View
            - Cart
            - Purchase
            - Remove from cart
- **product_id**: Unique number to identify the product in the event
- **category_id**: Unique number to identify the category of the product
- **category_code**: Stores primary and secondary categories of the product
- **brand**: Brand associated with the product
- **price**: Price of the product
- **user_id**: Unique ID for a customer
- **user_session**: Session ID for a user


### Initialising the SparkSession

The dataset provided is 5 GBs in size. Therefore, it is expected that you increase the driver memory to a greater number.

In [3]:
# Spark environment
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [4]:
# initialising the session with 14 GB driver memory
MAX_MEMORY = "14G"

spark = SparkSession \
    .builder \
    .appName("demo") \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

spark

In [5]:
# Spark session with 14 GB driver memory
spark.sparkContext.getConf().get('spark.driver.memory')

'14G'

In [6]:
# loading the dataset from the EC2 instance - 2019-Oct.csv
df = spark.read.csv('inputdata/2019-Oct.csv',header=True)

In [7]:
# exploring the dataframe - top 20 rows
df.show(20)

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                null|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|  33.20|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    null| 543.10|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:...|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:...|      view|   1004237|205301355563188265

In [8]:
# exploring the dataframe - schema
df.printSchema()

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_session: string (nullable = true)



In [9]:
# Number of rows and columns in the dataset
print((df.count(), len(df.columns)))

(42448764, 9)


So, we have a huge dataset that contains 42 million rows.

Now, tha dataset is loaded in Spark environment. Let's proceed with the desired tasks.

In [10]:
#Load Only Fraction of Dataset
# df = df.sample(False, 0.01, seed=42).collect()

In [11]:
# df = spark.createDataFrame(df)

In [12]:
# df.toPandas()

In [13]:
# print((df.count(), len(df.columns)))

## Task 1 - Data Exploration

**Find**
- 5 most popular:
    - Products sold by the e-commerce company in the month
    - Brands on the platform
    - Product categories
- Number of unique users and the most active user on the platform
- Average and maximum price for smartphones purchased by the customers
<br>

**Plot and comment**
- Event-type funnel distribution in e-commerce shopping journey
- Traffic on different days of the week

Provide the results in a separate report.

In [14]:
# 5 most popular products sold
from pyspark.sql.functions import *
import pyspark.sql.functions as F

# df.groupby('product_id').count().sort(desc('count')).show(5)
df.groupby('product_id').count().orderBy('count', ascending=False).show(5)

+----------+------+
|product_id| count|
+----------+------+
|   1004856|500354|
|   1004767|438232|
|   1005115|355786|
|   1004833|237545|
|   1004249|231070|
+----------+------+
only showing top 5 rows



In [15]:
# 5 most popular brands
# df.groupby('brand').count().sort(desc('count')).show(5)
df.groupby('brand').count().orderBy('count', ascending=False).show(5)

+-------+-------+
|  brand|  count|
+-------+-------+
|   null|6113008|
|samsung|5282775|
|  apple|4122554|
| xiaomi|3083763|
| huawei|1111205|
+-------+-------+
only showing top 5 rows



In [16]:
# Number of unique users
df.select(F.countDistinct("user_id")).show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                3022290|
+-----------------------+



In [17]:
# The most active user on the platform
# df.groupby('user_id','user_session').count().sort(desc('count')).show(10)
df.groupby('user_id','user_session').count().orderBy('count', ascending=False).show(5)

+---------+--------------------+-----+
|  user_id|        user_session|count|
+---------+--------------------+-----+
|513605798|fb075266-182d-4c1...| 1159|
|542048657|cfb90a35-9575-495...| 1137|
|513851612|2183f046-46f1-4ff...|  584|
|532769022|b2101293-44c1-481...|  564|
|566165785|e9d2b8ad-3e69-47f...|  425|
+---------+--------------------+-----+
only showing top 5 rows



In [18]:
# Average and Maximum price for smartphones purchased by the customers
# dfp = df.toPandas()
# dfp[(dfp['event_type'] == 'purchase') & (dfp['category_code'].str.contains('smartphone'))]['price'].astype(float).describe()

smartphonesdf = df.filter(col("category_code").contains("smartphone")).filter(col("event_type")=="purchase") 
smartphonesdf.groupBy('category_code').agg(F.mean('price'), F.max('price')).show()

+--------------------+-----------------+----------+
|       category_code|       avg(price)|max(price)|
+--------------------+-----------------+----------+
|electronics.smart...|464.6191130945664|    999.77|
+--------------------+-----------------+----------+



In [19]:
# Event-type funnel distribution
# Remember Spark dataframe cannot be visualised directly
import plotly.express as px

et = df.groupBy("event_type").count().orderBy('count', ascending=False).toPandas()
fig = px.funnel(et, x='count', y='event_type')
fig.show()

In [20]:
# Traffic on different days of the week
# Remember Spark dataframe cannot be visualised directly
dow = df.select(col('event_time'),dayofweek(col("event_time")).alias("dayofweek"))
dow_cnt = dow.groupby('dayofweek').count().toPandas()
fig = px.bar(dow_cnt, x='dayofweek', y='count')
fig.show()

## Task 2 - Feature Engineering

- Handle missing values (provide justification for approach)
- Generate the category code at 2 levels (Split into 2 columns)
    - Example: electronics.video.tv - electronics, video
- Capture user activity in different columns
    - Total activities (view/cart/etc.) in the session
    - Affinity towards a particular product (Product count for user)
    - Affinity towards a category (Secondary category count for user)
    - Average shopping expense for a product category (secondary)
    - Number of user sessions
- Impact of time: Day and Hour (Binning hours into 4 buckets)
- Reduction in brands for analysis: Top 20 + ‘others’
- Target variable generation: is_purchased


In [21]:
# Handling missing values 
# df.toPandas().isna().sum()
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).toPandas()

event_time  event_type  product_id  category_id  category_code    brand  \
0           0           0           0            0       13515609  6113008   

   price  user_id  user_session  
0      0        0             2

Reason for the action performed:



- ignore/drop rows with no user session
- use category_id instead of category_code as they both provide the same information
- keep the brand as null as 31% of brand column has null, imputing it will add bias to data and it would be wrong.

- we cant drop any rows from category_code or brand as it will result in loss of more than a quarter of the data

In [25]:
#Checking for duplicate entries
# if df.count() > df.dropDuplicates(df.columns).count():
#     print("DataFrame has Duplicates")

In [26]:
# Take required action here
# df = df.dropDuplicates(df.columns).collect()
# df = spark.createDataFrame(df)
# df = df.distinct()

In [27]:
# Generating 2 columns from category code
# Columns have only single entity (electronics.video.tv: electronics, video)
# Check if split command works with '.' or Spark treats it as special character
split_col = F.split(df['category_code'], '\.')
df = df.withColumn('category', split_col.getItem(0))
df = df.withColumn('secondary_category', split_col.getItem(1))

In [28]:
# Check if the column is added
df.show()

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+-----------+------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|   category|secondary_category|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+-----------+------------------+
|2019-10-01 00:00:...|      view|  44600062|2103807459595387724|                null|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|       null|              null|
|2019-10-01 00:00:...|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|  33.20|554748717|9333dfbd-b87a-470...| appliances|       environment|
|2019-10-01 00:00:...|      view|  17200506|2053013559792632471|furniture.living_...|    null| 543.10|519107250|566511c2-e2e3-422...|  furniture|       living_room|
|2019-10-0

In [29]:
from pyspark.sql.window import Window

In [30]:
# Activities in a session by the user
# Window functions can be helpful here

my_window = Window.partitionBy('user_id', 'user_session')

df = df.withColumn('noOfActivities', F.count('event_type').over(my_window))

df.show(7)

+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+-----------+------------------+--------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|   category|secondary_category|noOfActivities|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+-----------+------------------+--------------+
|2019-10-31 15:01:...|      view|   3701232|2053013565983425517|appliances.enviro...|  makita|328.81|286192318|3c48b80c-8cd9-4c2...| appliances|       environment|             1|
|2019-10-18 05:33:...|      view|   6000161|2053013560807654091|auto.accessories....|starline|244.79|286655746|912e74ac-dd84-476...|       auto|       accessories|             1|
|2019-10-02 12:48:...|      view|   1004741|2053013555631882655|electronics.smart...|  xiaomi|185.69|2939

In [31]:
# View count for a product by the user
# Window functions can be helpful here

windowPartitionProdCount  = Window.partitionBy("user_id")

df.withColumn('product_count',count(col('product_id')).over(windowPartitionProdCount)).show()

+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+-----------+------------------+--------------+-------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|   category|secondary_category|noOfActivities|product_count|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+-----------+------------------+--------------+-------------+
|2019-10-12 10:21:...|      view|   1004957|2053013555631882655|electronics.smart...|  xiaomi|341.74|292006376|9306d913-79c1-41e...|electronics|        smartphone|             1|            1|
|2019-10-01 10:32:...|      view|  23300385|2053013561956893455|                null|    arua| 34.11|318145786|e148e1ce-5032-4e9...|       null|              null|             2|            2|
|2019-10-01 10:33:...|      view|  

In [32]:
# View count for the secondary category by the user
# Window functions can be helpful here

windowPartitionSecCatCount  = Window.partitionBy("user_id")

df.withColumn('secondry_category_count',count(col('secondary_category')).over(windowPartitionSecCatCount)).show()

+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+-----------+------------------+--------------+-----------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|   category|secondary_category|noOfActivities|secondry_category_count|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+-----------+------------------+--------------+-----------------------+
|2019-10-12 10:21:...|      view|   1004957|2053013555631882655|electronics.smart...|  xiaomi|341.74|292006376|9306d913-79c1-41e...|electronics|        smartphone|             1|                      1|
|2019-10-01 10:32:...|      view|  23300385|2053013561956893455|                null|    arua| 34.11|318145786|e148e1ce-5032-4e9...|       null|              null|             2|          

In [33]:
# Session count for a user
# Window functions can be helpful here
df.groupby('user_id','user_session').count().show()

+---------+--------------------+-----+
|  user_id|        user_session|count|
+---------+--------------------+-----+
|513714706|46db4ccb-9e24-4a1...|    1|
|513733456|16d54a72-0afc-4fb...|    6|
|512553095|404a4a56-3aac-408...|    4|
|546370010|f17c0638-ba23-468...|    5|
|549811725|ff5dad2d-1418-47c...|    2|
|545803882|90222c7f-e14b-472...|   11|
|524102815|e7382057-39be-476...|    2|
|555129192|65468bc2-5ce0-4de...|    9|
|522959877|f440487c-9a86-452...|    1|
|551456160|c6ea65c5-86c7-44b...|    5|
|553428874|268eac90-33d5-475...|    1|
|513194193|4bd689a1-d939-405...|   13|
|537112281|26d05444-5c44-4ef...|    2|
|515256034|583dddf1-9343-4ce...|    4|
|551352669|ea650606-ae00-4e9...|    4|
|530487829|21234c6a-9cbd-43a...|    3|
|554291657|a749db7b-c9d5-4bc...|    5|
|519589854|da366db2-c0e4-4dd...|    2|
|522637243|bea6f7d5-9a7d-421...|    1|
|520744547|69f99b06-b514-499...|    1|
+---------+--------------------+-----+
only showing top 20 rows



In [34]:
from pyspark.sql.types import *

In [35]:
# Generating the hour variable
df = df.withColumn("timestamp",to_timestamp("event_time",'yyyy-MM-dd HH:mm:ss'))

df = df.withColumn('weekday', F.date_format('timestamp', 'EEEE'))
df = df.withColumn("hour", hour(col("timestamp")))
df.show()

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+-----------+------------------+--------------+-------------------+---------+----+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|   category|secondary_category|noOfActivities|          timestamp|  weekday|hour|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+-----------+------------------+--------------+-------------------+---------+----+
|2019-10-31 15:01:...|      view|   3701232|2053013565983425517|appliances.enviro...|  makita| 328.81|286192318|3c48b80c-8cd9-4c2...| appliances|       environment|             1|2019-10-31 15:01:46| Thursday|  15|
|2019-10-18 05:33:...|      view|   6000161|2053013560807654091|auto.accessories....|starline| 244.79|286655746|912e74ac-dd84-476...|       

In [36]:
def browsing_hour(hour):
    if (hour >= 5) and (hour < 12): return "Morning"
    elif (hour >= 12) and (hour < 4): return "Afternoon"
    elif (hour >= 4) and (hour < 8): return "Evening"
    else: return "Night"

In [37]:
bucket_df = udf(browsing_hour, StringType())
df = df.withColumn("hour_bucket", bucket_df("hour"))
df.show()

+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+-----------+------------------+--------------+-------------------+---------+----+-----------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|   category|secondary_category|noOfActivities|          timestamp|  weekday|hour|hour_bucket|
+--------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+-----------+------------------+--------------+-------------------+---------+----+-----------+
|2019-10-31 15:01:...|      view|   3701232|2053013565983425517|appliances.enviro...|  makita| 328.81|286192318|3c48b80c-8cd9-4c2...| appliances|       environment|             1|2019-10-31 15:01:46| Thursday|  15|      Night|
|2019-10-18 05:33:...|      view|   6000161|2053013560807654091|auto.accessories....|starlin

In [38]:
# Reduction in brands for analysis: Top 20 + ‘others'
# df.groupBy("brand").count().sort(desc("count")).show()
df.groupBy("brand").count().orderBy('count', ascending=False).show()

+--------+-------+
|   brand|  count|
+--------+-------+
|    null|6113008|
| samsung|5282775|
|   apple|4122554|
|  xiaomi|3083763|
|  huawei|1111205|
| lucente| 655861|
|      lg| 562404|
|   bosch| 557090|
|    oppo| 482887|
|    sony| 456644|
|    acer| 428153|
|cordiant| 368277|
|   artel| 338203|
|  lenovo| 338103|
|      hp| 305242|
| indesit| 290101|
|dauscher| 289853|
| philips| 282699|
| redmond| 279734|
| respect| 271167|
+--------+-------+
only showing top 20 rows



In [39]:
topBrands = ["samsung", "apple", "xiaomi", "huawei", "lucente", "lg", 
            "bosch", "oppo", "sony", "acer", "cordiant", "artel", "lenovo", 
            "hp", "indesit","dauscher", "philips", "redmond", "respect", "casio"]

In [40]:
df = df.withColumn('brand', when(df.brand.isin(topBrands), df.brand).otherwise('others'))

### Dropping the event_type "view" since probelm statement is to predict if a user will buy a product that is present in their cart

In [41]:
print(df.count())
df = df.where(df.event_type != 'view')
print(df.count())

42448764
1669365


In [42]:
# Generating 'is_purchased' variable
df = df.withColumn('is_purchased',when(df.event_type=="purchase" ,1).otherwise(0))

In [43]:
# Dropping redundant rows
# df = df.distinct()
# df.count()

In [44]:
# Printing the top 20 rows as output
df.show()

+--------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+------------+------------------+--------------+-------------------+---------+----+-----------+------------+
|          event_time|event_type|product_id|        category_id|       category_code|  brand| price|  user_id|        user_session|    category|secondary_category|noOfActivities|          timestamp|  weekday|hour|hour_bucket|is_purchased|
+--------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+------------+------------------+--------------+-------------------+---------+----+-----------+------------+
|2019-10-01 15:12:...|      cart|   1004856|2053013555631882655|electronics.smart...|samsung|132.87|441259588|69e19a80-b184-400...| electronics|        smartphone|             4|2019-10-01 15:12:23|  Tuesday|  15|      Night|           0|
|2019-10-17 03:32:...|      cart|   1004767|

In [45]:
# Dropping the redundant columns 
del_cols =["event_time","category_id","category_code","user_session","timestamp","hour",'noOfActivities']
df = df.drop(*del_cols)

In [46]:
# Columns in df after feature engineering
df.columns

['event_type',
 'product_id',
 'brand',
 'price',
 'user_id',
 'category',
 'secondary_category',
 'weekday',
 'hour_bucket',
 'is_purchased']

In [47]:
df.printSchema()

root
 |-- event_type: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- secondary_category: string (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour_bucket: string (nullable = true)
 |-- is_purchased: integer (nullable = false)



In [48]:
# Dropping redundant rows
df = df.distinct()
df.count()

1245285

In [49]:
df.toPandas()

event_type product_id    brand    price    user_id     category  \
0             cart    1004835  samsung   230.89  532230882  electronics   
1         purchase    1801690  samsung   368.04  555467805  electronics   
2         purchase    1003306    apple   587.56  547952735  electronics   
3         purchase    1004870  samsung   286.86  555148121  electronics   
4         purchase   26402505   others   950.73  555463488         None   
...            ...        ...      ...      ...        ...          ...   
1245280   purchase   10400392   others    25.71  566267483         kids   
1245281   purchase    1004566   huawei   164.84  541238676  electronics   
1245282       cart    1005115    apple   949.47  566272904  electronics   
1245283   purchase    1304409    apple  1402.87  562780507    computers   
1245284   purchase    1004767  samsung   242.63  551781196  electronics   

        secondary_category   weekday hour_bucket  is_purchased  
0               smartphone   Tuesday       Night             0  
1                    video   Tuesday       Night             1  
2               smartphone   Tuesday       Night             1  
3               smartphone   Tuesday       Night             1  
4                     None   Tuesday       Night             1  
...                    ...       ...         ...           ...  
1245280               toys  Thursday       Night             1  
1245281         smartphone  Thursday       Night             1  
1245282         smartphone  Thursday       Night             0  
1245283           notebook  Thursday       Night             1  
1245284         smartphone  Thursday       Night             1  

[1245285 rows x 10 columns]

In [51]:
# Storing the cleaned df in the instance to prevent repetition of steps again
# Rename the file based on your preferences
df.coalesce(1).write.option("header", "true").parquet("final_df__")

Proceed to another notebooks after saving the dataframe.